In [2]:
import os
import math
import datetime
import numpy as np
import pandas as pd


import time
import os.path

## step 1 obtain OSM network

In [3]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og

net = og.getNetFromOSMFile('tmc.osm',network_type=(network_type), default_lanes=True, default_speed=True)

og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


## step 2 Convert tmc Data into node and link

In [4]:
"Convert TMC Data into GMNS Format"
tmc = pd.read_csv('TMC_Identification.csv')

In [5]:

'''build node.csv'''
print('converting tmc data into gmns format...')
p=1

node_tmc = pd.DataFrame()
node_tmc['name'] = None
node_tmc['x_coord'] = None
node_tmc['y_coord'] = None
node_tmc['z_coord'] = None
node_tmc['node_type'] = None
node_tmc['ctrl_type'] = None
node_tmc['zone_id'] = None
node_tmc['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' nodes completed!')
        p = p + 1

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc.index.name = 'node_id'

node_tmc.index += 100000001 #index from 0

node_tmc.to_csv('node_tmc.csv')
print('node_tmc.csv (' + str(len(node_tmc)) + ' nodes' + ') generated!')

converting tmc data into gmns format...
10% nodes completed!
20% nodes completed!
30% nodes completed!
40% nodes completed!
50% nodes completed!
60% nodes completed!
70% nodes completed!
80% nodes completed!
90% nodes completed!
node_tmc.csv (64 nodes) generated!


In [6]:
node_tmc

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,geometry
node_id,,,,,,,,
100000001,110N05711,-77.3715,39.0144,None,tmc_start,None,None,POINT (-77.3715 39.0144)
100000002,110-05710,-77.3709,39.0141,None,tmc_start,None,None,POINT (-77.3709 39.0141)
100000003,110N05710,-77.3684,39.0127,None,tmc_start,None,None,POINT (-77.3684 39.0127)
100000004,110-05709,-77.3605,39.0086,None,tmc_start,None,None,POINT (-77.3605 39.0086)
100000005,110-05708,-77.3502,39.0028,None,tmc_start,None,None,POINT (-77.3502 39.0028)
...,...,...,...,...,...,...,...,...
100000060,110+05710,-77.3500,39.0030,None,tmc_start,None,None,POINT (-77.35 39.003)
100000061,110P05710,-77.3609,39.0090,None,tmc_start,None,None,POINT (-77.3609 39.009)
100000062,110+05711,-77.3673,39.0125,None,tmc_start,None,None,POINT (-77.3673 39.0125)


In [7]:
'''build link_tmc.csv'''
p = 1
link_tmc = pd.DataFrame()
link_tmc['name'] = None
link_tmc['corridor_id'] = None
link_tmc['corridor_link_order'] = None
link_tmc['from_node_id'] = None
link_tmc['to_node_id'] = None
link_tmc['directed'] = None
link_tmc['geometry_id'] = None
link_tmc['geometry'] = None
link_tmc['dir_flag'] = None
link_tmc['length'] = None
link_tmc['grade'] = None
link_tmc['facility_type'] = None
link_tmc['capacity'] = None
link_tmc['free_speed'] = None
link_tmc['lanes'] = None

for i in range(0,len(tmc)):
    link_tmc = link_tmc.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                'from_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' links completed!')
        p = p + 1
        
link_tmc.index.name = 'link_id'
link_tmc.index += 100000001


link_tmc.to_csv('link_tmc.csv')

print('link_tmc.csv (' + str(len(link_tmc)) + ' links' + ') generated!')

10% links completed!
20% links completed!
30% links completed!
40% links completed!
50% links completed!
60% links completed!
70% links completed!
80% links completed!
90% links completed!
link_tmc.csv (62 links) generated!


In [8]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,0.421999,None,None,None,None,None


In [9]:
## reading by detid
reading = pd.read_csv('Reading.csv')
reading = reading.loc[0:5000]


reading_dict = {}
gp = reading.groupby('tmc_code')
for key, form in gp:
    reading_dict[key] = {
        'measurement_tstamp':form['measurement_tstamp'].tolist(),
        'speed':form['speed'].tolist()
        }
time_period_reading = int(form['measurement_tstamp'].tolist()[1][-5:-3]) - int(form['measurement_tstamp'].tolist()[0][-5:-3]) 

In [10]:
'''build measurement_tmc.csv''' 

measurement_tmc = pd.DataFrame()
measurement_tmc['link_id_tmc'] = None
measurement_tmc['corridor_id'] = None
measurement_tmc['corridor_link_order'] = None
measurement_tmc['from_node_id'] = None
measurement_tmc['to_node_id'] = None
measurement_tmc['time_period'] = None
measurement_tmc['date'] = None
measurement_tmc['geometry'] = None
measurement_tmc['volume'] = None
measurement_tmc['travel_time'] = None
measurement_tmc['speed'] = None
measurement_tmc['reference_speed'] = None
measurement_tmc['density'] = None
measurement_tmc['queue'] = None
measurement_tmc['notes'] = None


k=0
p=1
measurement_tmc_dict = {}
for i in link_tmc.index:
    try:
        reading_dict_selected = reading_dict[link_tmc['name'][i]]
        for j in range(0,len(reading_dict_selected['measurement_tstamp']),int(15/time_period_reading)):
            measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'time_period': reading_dict_selected['measurement_tstamp'][j][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j][14:16]+'_'+\
                                                reading_dict_selected['measurement_tstamp'][j+int(15/time_period_reading)][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j+int(15/time_period_reading)][14:16],\
                                            'date': reading_dict_selected['measurement_tstamp'][j][:10],\
                                            'geometry': link_tmc['geometry'][i],\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed':round(np.mean(reading_dict_selected['speed'][j:j+int(15/time_period_reading)])),\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }
            k += 1

    except:
        measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                        'corridor_id': link_tmc['corridor_id'][i],\
                                        'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                        'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                        'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                        'time_period': None,\
                                        'date': None,\
                                        'geometry': link_tmc['geometry'][i],\
                                        'volume': None,\
                                        'travel_time': None,\
                                        'speed': None,\
                                        'reference_speed': None,\
                                        'density': None,\
                                        'queue': None,\
                                        'notes': None }
        k += 1

    if link_tmc.index.get_loc(i) > p/10 * len(link_tmc): 
        print(str(p*10)+"%"+' measurement_tmc completed!')
        p = p + 1

measurement_tmc = pd.DataFrame(measurement_tmc_dict).transpose()
measurement_tmc = measurement_tmc.dropna(subset=['time_period']) #remove na at the end of day of unrecorded ones
measurement_tmc = measurement_tmc.reset_index()
measurement_tmc = measurement_tmc.drop(['index'], 1)
measurement_tmc.to_csv('measurement_tmc.csv',index = False)
print('measurement_tmc.csv generated!')

10% measurement_tmc completed!
20% measurement_tmc completed!
30% measurement_tmc completed!
40% measurement_tmc completed!
50% measurement_tmc completed!
60% measurement_tmc completed!
70% measurement_tmc completed!
80% measurement_tmc completed!
90% measurement_tmc completed!
measurement_tmc.csv generated!


In [11]:
measurement_tmc

,link_id_tmc,corridor_id,corridor_link_order,from_node_id,to_node_id,time_period,date,geometry,volume,travel_time,speed,reference_speed,density,queue,notes
0,100000001,VA-7_EASTBOUND,38,100000001,100000002,0000_0015,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
1,100000001,VA-7_EASTBOUND,38,100000001,100000002,0015_0030,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
2,100000001,VA-7_EASTBOUND,38,100000001,100000002,0030_0045,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
3,100000001,VA-7_EASTBOUND,38,100000001,100000002,0045_0100,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
4,100000001,VA-7_EASTBOUND,38,100000001,100000002,0100_0115,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,100000061,VA-7_WESTBOUND,48,100000062,100000063,0530_0545,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,46,None,None,None,None
1634,100000061,VA-7_WESTBOUND,48,100000062,100000063,0545_0600,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,51,None,None,None,None
1635,100000061,VA-7_WESTBOUND,48,100000062,100000063,0600_0615,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,38,None,None,None,None
1636,100000061,VA-7_WESTBOUND,48,100000062,100000063,0615_0630,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,37,None,None,None,None


## step 3 mapping osm network to tmc network

In [12]:
link_base = pd.read_csv('link.csv', low_memory=False)
link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)

In [13]:
link_base

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,NaN,28,5966229,9582,14365,1,149.790510,1,59,NaN,motorway,1,"LINESTRING (-77.1825104 38.9725558, -77.182093...",auto,0
1,Nutley Street,765,8151072,14929,11825,1,40.179942,3,39,NaN,secondary,4,"LINESTRING (-77.2679213 38.8829327, -77.267880...",auto,0
2,Nutley Street,766,8151072,11825,10203,1,105.784046,3,39,NaN,secondary,4,"LINESTRING (-77.2678193 38.8825802, -77.267746...",auto,0
3,Nutley Street,767,8151072,10203,12741,1,202.218605,3,39,NaN,secondary,4,"LINESTRING (-77.2676173 38.8816422, -77.267591...",auto,0
4,Nutley Street,768,8151072,12741,14360,1,14.667111,3,39,NaN,secondary,4,"LINESTRING (-77.2674066 38.8798317, -77.267405...",auto,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4844,Spring Hill Road,31139,942574263,13473,13106,1,4.688207,2,39,NaN,secondary,4,"LINESTRING (-77.2388083 38.9287844, -77.238754...",auto,1
4845,Spring Hill Road,31140,942574264,14198,13105,1,18.798910,2,39,NaN,secondary,4,"LINESTRING (-77.23925079999999 38.928705, -77....",auto,0
4846,Spring Hill Road,31141,942574265,13628,14198,1,13.354776,2,39,NaN,secondary,4,"LINESTRING (-77.2394041 38.9286915, -77.239352...",auto,0
4847,Spring Hill Road,31142,942574267,13627,13662,1,31.889777,2,39,NaN,secondary,4,"LINESTRING (-77.2394068 38.9288064, -77.239543...",auto,0


In [14]:
def LLs2Dist(lon1, lat1, lon2, lat2): #WGS84 transfer coordinate system to distance(meter) #xy #credit to xtHuang0927
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c * 1000
    return distance

In [15]:
matching_tmc2gmns_dict = {}
k = 0
p = 1
for i in range(len(link_base)):
    lon_list = []
    lat_list = [] 
    link_geometry_list = link_base.loc[i,'geometry'][12:-1].split(", ")
    for link_geometry in link_geometry_list:
        lon_list.append(float(link_geometry.split(" ")[0]))
        lat_list.append(float(link_geometry.split(" ")[1]))
    center_lon = np.mean(lon_list)
    center_lat = np.mean(lat_list)


    distance_list = []
    for j in link_tmc.index:
        lon_tmc_list = []
        lat_tmc_list = []
        link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
        for link_tmc_geometry in link_tmc_geometry_list:
            lon_tmc_list.append(float(link_tmc_geometry.split(" ")[0]))
            lat_tmc_list.append(float(link_tmc_geometry.split(" ")[1]))
        center_tmc_lon = np.mean(lon_tmc_list)
        center_tmc_lat = np.mean(lat_tmc_list)
        
        distance_list.append(LLs2Dist(center_lon, center_lat, center_tmc_lon, center_tmc_lat))
    nearest_index = distance_list.index(min(distance_list))

    matching_tmc2gmns_dict[k] = {'name_base':link_base['name'][i],\
                    'link_id_base':link_base['link_id'][i],\
                    'from_node_id_base':link_base['from_node_id'][i],\
                    'to_node_id_base':link_base['to_node_id'][i],\
                    'category_id_base':link_tmc.index.get_loc(link_tmc.iloc[[nearest_index]].index[0])+1,\
                    'geometry_base':link_base['geometry'][i],\
                    'name_tmc':link_tmc.iloc[nearest_index]['name'],\
                    'link_id_tmc':int(link_tmc.iloc[[nearest_index]].index.values[0]),\
                    'from_node_id_tmc':int(link_tmc.iloc[nearest_index]['from_node_id']),\
                    'to_node_id_tmc':int(link_tmc.iloc[nearest_index]['to_node_id']),\
                    'category_id_tmc':link_tmc.index.get_loc(link_tmc.iloc[[nearest_index]].index[0])+1,\
                    'geometry_tmc':link_tmc.iloc[nearest_index]['geometry']}
    k += 1

    # if i % (len(link_base)/10) == 0 : print("{:.0%}".format(i/len(link_base))+' matching completed!')
    
    if i > p/10 * len(link_base): 
        print(str(p*10)+"%"+' matching completed!')
        p = p + 1
        

matching_tmc2gmns = pd.DataFrame(matching_tmc2gmns_dict).transpose()
matching_tmc2gmns.to_csv('matching_tmc2gmns.csv',index = False)
print('matching_tmc2gmns.csv generated!')

10% matching completed!
20% matching completed!
30% matching completed!
40% matching completed!
50% matching completed!
60% matching completed!
70% matching completed!
80% matching completed!
90% matching completed!
matching_tmc2gmns.csv generated!


In [16]:
matching_tmc2gmns


,name_base,link_id_base,from_node_id_base,to_node_id_base,category_id_base,geometry_base,name_tmc,link_id_tmc,from_node_id_tmc,to_node_id_tmc,category_id_tmc,geometry_tmc
0,NaN,28,9582,14365,45,"LINESTRING (-77.1825104 38.9725558, -77.182093...",110P05699,100000045,100000046,100000047,45,"LINESTRING (-77.2447 38.9323,-77.2487 38.9364)"
1,Nutley Street,765,14929,11825,23,"LINESTRING (-77.2679213 38.8829327, -77.267880...",110-05696,100000023,100000023,100000024,23,"LINESTRING (-77.2302 38.9178,-77.227 38.9158)"
2,Nutley Street,766,11825,10203,23,"LINESTRING (-77.2678193 38.8825802, -77.267746...",110-05696,100000023,100000023,100000024,23,"LINESTRING (-77.2302 38.9178,-77.227 38.9158)"
3,Nutley Street,767,10203,12741,23,"LINESTRING (-77.2676173 38.8816422, -77.267591...",110-05696,100000023,100000023,100000024,23,"LINESTRING (-77.2302 38.9178,-77.227 38.9158)"
4,Nutley Street,768,12741,14360,23,"LINESTRING (-77.2674066 38.8798317, -77.267405...",110-05696,100000023,100000023,100000024,23,"LINESTRING (-77.2302 38.9178,-77.227 38.9158)"
...,...,...,...,...,...,...,...,...,...,...,...,...
4844,Spring Hill Road,31139,13473,13106,44,"LINESTRING (-77.2388083 38.9287844, -77.238754...",110+05699,100000044,100000045,100000046,44,"LINESTRING (-77.2365 38.924,-77.2447 38.9323)"
4845,Spring Hill Road,31140,14198,13105,44,"LINESTRING (-77.23925079999999 38.928705, -77....",110+05699,100000044,100000045,100000046,44,"LINESTRING (-77.2365 38.924,-77.2447 38.9323)"
4846,Spring Hill Road,31141,13628,14198,44,"LINESTRING (-77.2394041 38.9286915, -77.239352...",110+05699,100000044,100000045,100000046,44,"LINESTRING (-77.2365 38.924,-77.2447 38.9323)"
4847,Spring Hill Road,31142,13627,13662,44,"LINESTRING (-77.2394068 38.9288064, -77.239543...",110+05699,100000044,100000045,100000046,44,"LINESTRING (-77.2365 38.924,-77.2447 38.9323)"


## step 4 generating measurement_osm 

In [17]:
link_base = pd.read_csv('link.csv', low_memory=False)
link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)

'''build measurement_base.csv''' 


measurement_base = pd.DataFrame()
measurement_base['link_id'] = None
measurement_base['osm_way_id'] = None
measurement_base['from_node_id'] = None
measurement_base['to_node_id'] = None
measurement_base['lanes'] = None
measurement_base['length'] = None
measurement_base['time_period'] = None
measurement_base['date'] = None
measurement_base['geometry'] = None
measurement_base['volume'] = None
measurement_base['speed'] = None
measurement_base['ip_address'] = None

k=0
p=1
measurement_base_dict = {}
for i in link_base.index:
    try:
        measurement_tmc_selected = measurement_tmc[measurement_tmc['link_id_tmc'] == matching_tmc2gmns['link_id_tmc'][i]]
        for j in measurement_tmc_selected.index:
            measurement_base_dict[k] = {'link_id': link_base['link_id'][i],\
                                            'osm_way_id':link_base['osm_way_id'][i],\
                                            'from_node_id': link_base['from_node_id'][i],\
                                            'to_node_id': link_base['to_node_id'][i],\
                                            'lanes': link_base['lanes'][i], \
                                            'length': link_base['length'][i], \
                                            'time_period': measurement_tmc_selected['time_period'][j],\
                                            'date': measurement_tmc_selected['date'][j],\
                                            'geometry': link_base['geometry'][i],\
                                            'volume': None,\
                                            'speed': measurement_tmc_selected['speed'][j],\
                                            'ip_address': 'www.openstreetmap.org/?way=' + str(link_base['osm_way_id'][i])} 

            k += 1
    except:
        measurement_base_dict[k] = {'link_id': link_base['link_id'][i],\
                                            'osm_way_id':link_base['osm_way_id'][i],\
                                            'from_node_id': link_base['from_node_id'][i],\
                                            'to_node_id': link_base['to_node_id'][i],\
                                            'lanes': link_base['lanes'][i], \
                                            'length': link_base['length'][i], \
                                            'time_period':None,\
                                            'date': None,\
                                            'geometry': link_base['geometry'][i],\
                                            'volume': None,\
                                            'speed': None,\
                                            'ip_address': 'www.openstreetmap.org/?way=' + str(link_base['osm_way_id'][i])}

        k += 1
    
    if link_base.index.get_loc(i) > p/10 * len(link_base): 
        # measurement_base = pd.DataFrame(measurement_base_dict).transpose()
        # if p == 1:
        #     measurement_base.to_csv('measurement_base.csv',header=True,index = False)
        #     measurement_base_dict = {}
        # else:
        #     measurement_base.to_csv('measurement_base.csv',header=False,mode='a',index = False)
        #     measurement_base_dict = {}
        print(str(p*10)+"%"+' measurement_base completed!')
        p = p + 1

measurement_base = pd.DataFrame(measurement_base_dict).transpose()
measurement_base.to_csv('measurement_base.csv',index = False)
print('measurement_base.csv generated!')

10% measurement_base completed!
20% measurement_base completed!
30% measurement_base completed!
40% measurement_base completed!
50% measurement_base completed!
60% measurement_base completed!
70% measurement_base completed!
80% measurement_base completed!
90% measurement_base completed!
measurement_base.csv generated!


In [18]:
measurement_base

,link_id,osm_way_id,from_node_id,to_node_id,lanes,length,time_period,date,geometry,volume,speed,ip_address
0,28,5966229,9582,14365,1,149.791,0000_0015,2015-04-01,"LINESTRING (-77.1825104 38.9725558, -77.182093...",None,36,www.openstreetmap.org/?way=5966229
1,28,5966229,9582,14365,1,149.791,0015_0030,2015-04-01,"LINESTRING (-77.1825104 38.9725558, -77.182093...",None,36,www.openstreetmap.org/?way=5966229
2,28,5966229,9582,14365,1,149.791,0030_0045,2015-04-01,"LINESTRING (-77.1825104 38.9725558, -77.182093...",None,38,www.openstreetmap.org/?way=5966229
3,28,5966229,9582,14365,1,149.791,0045_0100,2015-04-01,"LINESTRING (-77.1825104 38.9725558, -77.182093...",None,37,www.openstreetmap.org/?way=5966229
4,28,5966229,9582,14365,1,149.791,0100_0115,2015-04-01,"LINESTRING (-77.1825104 38.9725558, -77.182093...",None,36,www.openstreetmap.org/?way=5966229
...,...,...,...,...,...,...,...,...,...,...,...,...
132926,31143,942574268,14199,13627,2,12.588,0545_0600,2015-04-01,"LINESTRING (-77.2392614 38.9288089, -77.239354...",None,40,www.openstreetmap.org/?way=942574268
132927,31143,942574268,14199,13627,2,12.588,0600_0615,2015-04-01,"LINESTRING (-77.2392614 38.9288089, -77.239354...",None,56,www.openstreetmap.org/?way=942574268
132928,31143,942574268,14199,13627,2,12.588,0615_0630,2015-04-01,"LINESTRING (-77.2392614 38.9288089, -77.239354...",None,29,www.openstreetmap.org/?way=942574268
132929,31143,942574268,14199,13627,2,12.588,0630_0645,2015-04-01,"LINESTRING (-77.2392614 38.9288089, -77.239354...",None,33,www.openstreetmap.org/?way=942574268


In [22]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
